In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd

import utm
import time
import os.path

import shapely
import shapely.geometry as geom
import geopandas as gpd

from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.geometry import Point

In [2]:
def getDegree(latA, lonA, latB, lonB):
    """
    Args:
        point p1(latA, lonA)
        point p2(latB, lonB)
    Returns:
        bearing between the two GPS points,
        default: the basis of heading direction is north
    """
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
    dLon = radLonB - radLonA
    y = math.sin(dLon) * math.cos(radLatB)
    x = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = np.degrees(math.atan2(y, x))
    brng = (brng + 360) % 360
    # brng = 360 - brng
    return brng

def bearing(geometry):
   x, y = shapely.wkt.loads(geometry).coords.xy
   xy = pd.DataFrame({'LON':x,'LAT':y})
   return getDegree(xy.head(1)['LAT'], xy.head(1)['LON'], xy.tail(1)['LAT'], xy.tail(1)['LON'])


In [3]:
link_base = pd.read_csv('link_base.csv', low_memory=False)
link_base['bearing_angle'] = link_base['geometry'].apply(bearing)
link_base

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway,is_link,VDF_FFTT1,VDF_cap1,bearing_angle
0,NaN,0,5565454,1655,533,1,238.513115,1,NaN,NaN,motorway,1,"LINESTRING (-112.1148898 33.6420933, -112.1149...",auto,0,1,NaN,NaN,353.631857
1,NaN,1,5567209,526,529,1,288.986649,2,NaN,NaN,motorway,1,"LINESTRING (-112.1159268 33.6346955, -112.1157...",auto,0,1,NaN,NaN,7.463857
2,East Deer Valley Road,2,5569117,90071,3084,1,139.096981,2,NaN,NaN,secondary,4,"LINESTRING (-111.9601649 33.6763071, -111.9602...",auto,0,0,NaN,NaN,271.651179
3,NaN,3,5573783,5,3617,1,239.843469,1,NaN,NaN,motorway,1,"LINESTRING (-112.1154567 33.6442126, -112.1155...",auto,0,1,NaN,NaN,190.829480
4,NaN,4,5575041,468,748,1,201.561269,2,NaN,NaN,motorway,1,"LINESTRING (-112.1112736 33.6851721, -112.1115...",auto,0,1,NaN,NaN,337.005302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4944,East Pinnacle Peak Road,90044,218355970,90022,90023,1,63.619476,2,40.0,NaN,residential,6,"LINESTRING (-111.9215591 33.6987582, -111.9216...",auto,0,0,NaN,NaN,270.080305
4945,East Pinnacle Peak Road,90045,520542430,90023,90024,1,46.403599,2,40.0,NaN,residential,6,"LINESTRING (-111.9222468 33.698759, -111.92244...",auto,0,0,NaN,NaN,270.137435
4946,East Pinnacle Peak Road,90046,529967240,90024,3231,1,50.567017,2,40.0,NaN,residential,6,"LINESTRING (-111.9227484 33.69876, -111.923295...",auto,0,0,NaN,NaN,270.327731
4947,North Scottsdale Road,90047,531436240,3442,3443,1,7.405588,3,40.0,NaN,residential,6,"LINESTRING (-111.9253207 33.699241, -111.92532...",auto,1,0,NaN,NaN,359.928427


In [4]:
multiline_string_base_list = []
multiline_string_base_list_sub = []
for j in link_base.index:
    link_base_geometry_list = link_base.loc[j,'geometry'][12:-1].split(", ")
    for link_base_geometry in link_base_geometry_list:
        multiline_string_base_list_sub.append((float(link_base_geometry.split(" ")[0]),float(link_base_geometry.split(" ")[1])))
    multiline_string_base_list_sub = tuple(multiline_string_base_list_sub)
    multiline_string_base_list.append(multiline_string_base_list_sub)
    multiline_string_base_list_sub = []

from shapely.geometry import MultiLineString

line_base = MultiLineString(multiline_string_base_list) 

In [5]:
link_measurement_TMC = pd.read_csv('link_measurement_TMC.csv', low_memory=False)
link_TMC = pd.read_csv('link_TMC.csv', low_memory=False)
link_TMC = link_TMC[link_TMC['link_id'].isin(set(link_measurement_TMC['link_id']))]
link_TMC = link_TMC.reset_index()
link_TMC = link_TMC.drop(['index'], 1)


In [6]:
in_bbox_index_list = []
for i in link_TMC.index:
    link_tmc_geometry_list = link_TMC.loc[i,'geometry'][12:-1].split(", ")
    start_longitude = float(link_tmc_geometry_list[0].split(" ")[0])
    start_latitude = float(link_tmc_geometry_list[0].split(" ")[1])
    end_longitude = float(link_tmc_geometry_list[1].split(" ")[0])
    end_latitude = float(link_tmc_geometry_list[1].split(" ")[1])
    if (start_longitude > line_base.bounds[0]) & (start_longitude < line_base.bounds[2]) & \
         (end_longitude > line_base.bounds[0]) & (end_longitude < line_base.bounds[2]) & \
             (start_latitude > line_base.bounds[1]) & (start_latitude < line_base.bounds[3]) & \
         (end_latitude > line_base.bounds[1]) & (end_latitude < line_base.bounds[3]):
         in_bbox_index_list.append(i)

link_TMC = link_TMC.loc[in_bbox_index_list]
link_TMC = link_TMC.reset_index()
link_TMC = link_TMC.drop(['index'], 1)

link_TMC['bearing_angle'] = link_TMC['geometry'].apply(bearing)
link_TMC

,link_id,name,from_node_id,to_node_id,facility_type,link_type,dir_flag,length,lanes,free_speed,...,VDF_beta4,VDF_frequency4,VDF_FFTT5,VDF_cap5,VDF_alpha5,VDF_beta5,VDF_frequency5,geometry,Unnamed: 37,bearing_angle
0,20739AB,NaN,18281,18282,NaN,1,1,0.44804,3,41.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-112.195152 33.638579, -112.202916...",NaN,267.371410
1,20749AB,NaN,18299,18300,NaN,1,1,0.85350,3,71.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-112.195491 33.668369, -112.210285...",NaN,267.307085
2,20750AB,NaN,18301,18302,NaN,1,1,0.84076,3,71.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-112.210231 33.667568, -112.195658...",NaN,87.258114
3,20755AB,NaN,18308,18309,NaN,1,1,0.95895,1,75.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-112.210902 33.667642, -112.194277...",NaN,87.657056
4,20756AB,NaN,18310,18311,NaN,1,1,0.95571,1,75.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-112.194350 33.668297, -112.210918...",NaN,267.591851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,26896BA,NaN,21941,21946,NaN,1,1,0.59761,2,41.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-112.030768 33.691757, -112.025744...",NaN,28.866540
102,27025BA,NaN,20064,20015,NaN,1,1,0.27922,1,41.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-111.947630 33.698685, -111.942783...",NaN,90.225988
103,28049BA,NaN,22549,20019,NaN,1,1,0.26873,2,40.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-111.925333 33.655166, -111.929995...",NaN,269.085959
104,28060BA,NaN,22549,22613,NaN,1,1,0.11014,3,40.0,...,0.0,0,0.0,0.0,0.0,0.0,0,"LINESTRING (-111.925333 33.655166, -111.925356...",NaN,359.313610


In [9]:
from shapely.wkt import loads
matching_tmc2base_dict = {}
k = 0
p = 1
for i in link_TMC.index:
    angle_list = list(abs(link_TMC.loc[i,'bearing_angle']-list(link_base['bearing_angle'])))
    small_angle_list = [i for i, value in enumerate(angle_list) if value < 45]
    for dividing_rate in range(1,11):
        start_point_tmc = loads(link_TMC['geometry'][i]).interpolate((dividing_rate-1)/10, normalized=True)
        end_point_tmc = loads(link_TMC['geometry'][i]).interpolate(dividing_rate/10, normalized=True)
        distance_list = []
        
        for j in small_angle_list:
            distance_list.append(LineString([start_point_tmc, end_point_tmc]).distance(loads(link_base['geometry'][j])))
        df_distance = pd.DataFrame({'index':small_angle_list,'distance':distance_list})
        nearest_index = int(df_distance.loc[df_distance['distance'].idxmin()]['index'])

        matching_tmc2base_dict[k] = {'name_tmc':link_TMC.loc[i]['link_id'],\
                                        'from_node_id_tmc':link_TMC.loc[i]['from_node_id'],\
                                        'to_node_id_tmc':link_TMC.loc[i]['to_node_id'],\
                                        'bearing_angle_tmc':link_TMC.loc[i]['bearing_angle'],\
                                        'geometry_tmc':link_TMC.loc[i]['geometry'],\
                                        'name_base':link_base['name'][nearest_index],\
                                        'link_id_base':link_base['link_id'][nearest_index],\
                                        'from_node_id_base':link_base['from_node_id'][nearest_index],\
                                        'to_node_id_base':link_base['to_node_id'][nearest_index],\
                                        'geometry_base':link_base['geometry'][nearest_index],\
                                        'distance':min(distance_list),\
                                        'geometry_tmc_base':'MULTILINESTRING ('+ link_TMC.loc[i]['geometry'][11:] + \
                                                            ', ' + link_base['geometry'][nearest_index][11:]+')'}
        k += 1

        
    if link_TMC.index.get_loc(i) > p/10 * len(link_TMC): 
        print(str(p*10)+"%"+' matching completed!')
        p = p + 1
            

matching_tmc2base = pd.DataFrame(matching_tmc2base_dict).transpose()
matching_tmc2base_drop_duplicates = matching_tmc2base.drop(columns=['distance']).drop_duplicates()
matching_tmc2base_drop_duplicates['distance'] = matching_tmc2base.loc[matching_tmc2base_drop_duplicates.index]['distance']
matching_tmc2base_drop_duplicates = matching_tmc2base_drop_duplicates.reset_index()
matching_tmc2base_drop_duplicates = matching_tmc2base_drop_duplicates.drop(['index'], 1)
matching_tmc2base = matching_tmc2base_drop_duplicates

matching_tmc2base.to_csv('matching_tmc2base.csv',index = False)
print('matching_tmc2base.csv generated!')

10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2base.csv generated!


In [8]:
matching_tmc2base

,name_tmc,from_node_id_tmc,to_node_id_tmc,bearing_angle_tmc,geometry_tmc,name_base,link_id_base,from_node_id_base,to_node_id_base,geometry_base,geometry_tmc_base,distance
0,20739AB,18281,18282,267.371,"LINESTRING (-112.195152 33.638579, -112.202916...",West Bell Road,2080,2414,74,"LINESTRING (-112.1944975 33.6386541, -112.1950...","MULTILINESTRING ((-112.195152 33.638579, -112....",6.22212e-05
1,20739AB,18281,18282,267.371,"LINESTRING (-112.195152 33.638579, -112.202916...",West Bell Road,2088,74,2413,"LINESTRING (-112.1951793 33.6386404, -112.1953...","MULTILINESTRING ((-112.195152 33.638579, -112....",6.55249e-05
2,20739AB,18281,18282,267.371,"LINESTRING (-112.195152 33.638579, -112.202916...",West Bell Road,2059,2413,2412,"LINESTRING (-112.1988333 33.6385074, -112.1994...","MULTILINESTRING ((-112.195152 33.638579, -112....",6.89027e-05
3,20739AB,18281,18282,267.371,"LINESTRING (-112.195152 33.638579, -112.202916...",West Bell Road,2084,2412,2406,"LINESTRING (-112.1994119 33.6384845, -112.2001...","MULTILINESTRING ((-112.195152 33.638579, -112....",6.78932e-05
4,20739AB,18281,18282,267.371,"LINESTRING (-112.195152 33.638579, -112.202916...",West Bell Road,728,2405,90101,"LINESTRING (-112.2019525 33.6383814, -112.2021...","MULTILINESTRING ((-112.195152 33.638579, -112....",6.16212e-05
...,...,...,...,...,...,...,...,...,...,...,...,...
482,28049BA,22549,20019,269.086,"LINESTRING (-111.925333 33.655166, -111.929995...",East Mayo Boulevard,3644,1877,3054,"LINESTRING (-111.9275244 33.6552693, -111.9287...","MULTILINESTRING ((-111.925333 33.655166, -111....",0.000134511
483,28049BA,22549,20019,269.086,"LINESTRING (-111.925333 33.655166, -111.929995...",East Mayo Boulevard,3647,3055,90114,"LINESTRING (-111.9288374 33.6552713, -111.9294...","MULTILINESTRING ((-111.925333 33.655166, -111....",0.000155248
484,28060BA,22549,22613,359.314,"LINESTRING (-111.925333 33.655166, -111.925356...",NaN,3633,894,1797,"LINESTRING (-111.9235933 33.6550975, -111.9235...","MULTILINESTRING ((-111.925333 33.655166, -111....",0.00173678
485,28060BA,22549,22613,359.314,"LINESTRING (-111.925333 33.655166, -111.925356...",North Scottsdale Road,3613,599,3531,"LINESTRING (-111.925211 33.6587577, -111.92521...","MULTILINESTRING ((-111.925333 33.655166, -111....",0.00199897


In [54]:
from shapely.geometry import LineString
l1 = LineString([(42.073407, -87.806245), (42.0752508,-87.8080299)])
from shapely.wkt import loads
l2 = loads(" LINESTRING(35.442827 -79.470579, 35.444889 -79.469465, 35.445829 -79.468907, 35.446608 -79.468294, 35.447893 -79.46687)")
l1.distance(l2)
# 10.650628707489625
l2.distance(l1)
# 10.650628707489625

10.650628707489625

In [52]:
link_TMC.to_csv('xx_inner.csv')

In [25]:
utm.from_latlon(47.9941214, 7.8509671)

(414278.16730997514, 5316285.595228155, 32, 'T')

In [29]:
u = utm.from_latlon(47.9941214, 7.8509671)
utm.to_latlon(*u)

(47.994121402271354, 7.850967099948628)

In [30]:
u

(414278.16730997514, 5316285.595228155, 32, 'T')